# Safety Monitoring Dataset

In [1]:
# Importing the Lib
import pandas as pd
import os

safety_df = pd.read_csv("D:\Machine_Learning_Project\AI_for_Elderly_Care_and_Support\Dataset\safety_monitoring.csv")
safety_df.head()

<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
C:\Users\DELL\AppData\Local\Temp\ipykernel_15744\3876787320.py:5: SyntaxWarning: invalid escape sequence '\M'
  safety_df = pd.read_csv("D:\Machine_Learning_Project\AI_for_Elderly_Care_and_Support\Dataset\safety_monitoring.csv")


,Device-ID/User-ID,Timestamp,Movement Activity,Fall Detected (Yes/No),Impact Force Level,Post-Fall Inactivity Duration (Seconds),Location,Alert Triggered (Yes/No),Caregiver Notified (Yes/No),Unnamed: 9
0,D1000,1/7/2025 16:04,No Movement,No,-,0,Kitchen,No,No,NaN
1,D1001,1/20/2025 15:45,Lying,No,-,0,Living Room,No,No,NaN
2,D1002,1/2/2025 2:42,No Movement,No,-,0,Bedroom,No,No,NaN
3,D1003,1/1/2025 22:36,Lying,No,-,0,Kitchen,No,No,NaN
4,D1004,1/3/2025 16:30,No Movement,No,-,0,Bedroom,No,No,NaN


In [2]:
# Columns
print(safety_df.columns)

Index(['Device-ID/User-ID', 'Timestamp', 'Movement Activity',
       'Fall Detected (Yes/No)', 'Impact Force Level',
       'Post-Fall Inactivity Duration (Seconds)', 'Location',
       'Alert Triggered (Yes/No)', 'Caregiver Notified (Yes/No)',
       'Unnamed: 9'],
      dtype='object')


In [3]:
# Total Rows and Columns
print("Total Rows and Columns are", safety_df.shape)

Total Rows and Columns are (10000, 10)


In [4]:
# Checking if any missing value is there
safety_df.isnull().sum()

Device-ID/User-ID                              0
Timestamp                                      0
Movement Activity                              0
Fall Detected (Yes/No)                         0
Impact Force Level                             0
Post-Fall Inactivity Duration (Seconds)        0
Location                                       0
Alert Triggered (Yes/No)                       0
Caregiver Notified (Yes/No)                    0
Unnamed: 9                                 10000
dtype: int64

In [5]:
# Renaming the Columns
safety_df.columns = [
    'user_id',
    'timestamp',
    'movement_activity',
    'fall_detected',
    'impact_force_level',
    'post_fall_inactivity_duration',
    'location',
    'alert_triggered',
    'caregiver_notified',
    'extra_column'
]

safety_df.head(2)

,user_id,timestamp,movement_activity,fall_detected,impact_force_level,post_fall_inactivity_duration,location,alert_triggered,caregiver_notified,extra_column
0,D1000,1/7/2025 16:04,No Movement,No,-,0,Kitchen,No,No,NaN
1,D1001,1/20/2025 15:45,Lying,No,-,0,Living Room,No,No,NaN


In [6]:
# Droping the Unnecessary Columns
safety_df.drop(columns=['extra_column'], inplace = True)

In [7]:
# Converting yes/No columns to binary
yes_no_cols = ['fall_detected', 'alert_triggered', 'caregiver_notified']
safety_df[yes_no_cols] = safety_df[yes_no_cols].applymap(lambda x: 1 if str(x).strip().lower() == 'yes' else 0)


C:\Users\DELL\AppData\Local\Temp\ipykernel_15744\2990023784.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  safety_df[yes_no_cols] = safety_df[yes_no_cols].applymap(lambda x: 1 if str(x).strip().lower() == 'yes' else 0)


In [8]:
# Converting timestamp to datetime
safety_df["timestamp"] = pd.to_datetime(safety_df["timestamp"], errors = "coerce")

In [9]:
# Striping the whitespace from the columns
safety_df["movement_activity"] = safety_df["movement_activity"].str.strip()
safety_df["location"] = safety_df["location"].str.strip()

In [10]:
# Checking the missing value
missing_value = safety_df.isnull().sum() 

In [11]:
safety_df.head()

,user_id,timestamp,movement_activity,fall_detected,impact_force_level,post_fall_inactivity_duration,location,alert_triggered,caregiver_notified
0,D1000,2025-01-07 16:04:00,No Movement,0,-,0,Kitchen,0,0
1,D1001,2025-01-20 15:45:00,Lying,0,-,0,Living Room,0,0
2,D1002,2025-01-02 02:42:00,No Movement,0,-,0,Bedroom,0,0
3,D1003,2025-01-01 22:36:00,Lying,0,-,0,Kitchen,0,0
4,D1004,2025-01-03 16:30:00,No Movement,0,-,0,Bedroom,0,0


In [12]:
safety_df.isnull().sum()

user_id                          0
timestamp                        0
movement_activity                0
fall_detected                    0
impact_force_level               0
post_fall_inactivity_duration    0
location                         0
alert_triggered                  0
caregiver_notified               0
dtype: int64

In [13]:
# Saving the cleaned data to cleaned data folder
data_folder = "../cleaned_data"
file_name = "cleaned_safety_monitoring_dataset.csv"
file_path = os.path.join(data_folder, file_name)
safety_df.to_csv(file_path, index=False)

## Integrating the PostgreSQL

In [14]:
import psycopg2

# Connecting to postgresql

conn = psycopg2.connect(
    dbname = "elderly_care",
    user = "postgres",
    password = "system",
    host = "localhost",
    port = "5432"
    
)
cur = conn.cursor()

In [15]:
conn.rollback()

In [16]:
for _, row in safety_df.iterrows():
    cur.execute("""
        INSERT INTO safety_monitoring (
            user_id, timestamp, movement_activity, fall_detected,
            impact_force_level, post_fall_inactivity_duration,
            location, alert_triggered, caregiver_notified
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

In [17]:
conn.commit()
cur.close()
conn.close()


print("Safety monitoring data inserted in ")

Safety monitoring data inserted in 
